# 1. Introduction

This notebook focuses on training, evaluating, and selecting machine learning models for **loan default (credit risk) prediction**.

All data preprocessing steps—including exploratory data analysis, feature encoding, numerical scaling, and feature assembly—have already been completed in a previous notebook. This notebook assumes that the input features are fully numerical and ready for modeling.

The objective of this notebook is to experiment with multiple model families, compare their performance using appropriate evaluation metrics, and select the most suitable model for final evaluation on unseen data.

The modeling workflow is designed to ensure fair comparison between models and to prevent data leakage by keeping the test dataset completely isolated until the final evaluation stage.



# 2. Data Loading